In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from sklearn.model_selection import train_test_split

import keras
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,BatchNormalization
from keras.callbacks import EarlyStopping,LearningRateScheduler,ReduceLROnPlateau
from keras.optimizers import SGD,RMSprop
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

from keras.models import Sequential
warnings.filterwarnings('ignore')

from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.layers import Dropout

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def display_images(desired_shape, images, num_images=6):
    plt.figure(figsize=(12, 6))
    for i in range(num_images):
        ax = plt.subplot(2, 3, i + 1)
        plt.imshow(images[i].reshape(desired_shape[0], desired_shape[1]), cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

def process_pixel_data(pixel_data, desired_shape):
    return np.array([np.array([float(x) for x in item.split()]).reshape(desired_shape) for item in pixel_data])

def process_labels(labels):
    # Convert labels to numerical format if they're not numeric
    unique_labels = {label: idx for idx, label in enumerate(np.unique(labels))}
    return np.array([unique_labels[label] for label in labels]), unique_labels

def make_model(pixel_data, labels):
    # Process pixel data
    input_shape = (64, 64, 1) # Corresponds to target_size in preprocess.py, but with an added dimension
    processed_images = process_pixel_data(pixel_data, desired_shape=input_shape)

    # display_images(input_shape, processed_images)

    # Process labels
    processed_labels, label_map = process_labels(labels)
    processed_labels = to_categorical(processed_labels)

    # Splitting data into training and testing sets
    train_img, test_img, train_cancer_cat, test_cancer_cat = train_test_split(
        processed_images, processed_labels, test_size=0.2, random_state=42)
    
    # CNN Model
    model = Sequential([
        Conv2D(256, (3,3), activation='relu', padding='same', input_shape=input_shape),
        MaxPool2D(2,2),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPool2D(2,2),
        BatchNormalization(),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        MaxPool2D(2,2),
        BatchNormalization(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(64, activation='relu'),
        Dense(len(label_map), activation='softmax') 
    ])

    model.compile(optimizer=Adam(learning_rate=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_img, train_cancer_cat, validation_data=(test_img, test_cancer_cat), epochs=20)

    # Evaluation and reporting
    print("Training Accuracy:", history.history['accuracy'][-1])
    print("Validation Accuracy:", history.history['val_accuracy'][-1])

    # Evaluation
    print("Training accuracies:")
    print(history.history["accuracy"][-1])

    return model, label_map, history


In [18]:
import os
import subprocess
!find . -name ".DS_Store" -delete

# TODO: Need to make a for-loop that goes through every folder in images and annotations. This is 
# because the way the code works, it only goes through one folder at a time, so we need a loop
# that calls big_helper.py multiple times. I don't care about runtime anymore...

# NOTE: There will be Not-Found errors because not all the image files are downloaded even though all
# the annotations are downloaded.

# # Some of the stencil code uses depricated code, but it is still needed
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)

# This code block basically preprocesses the image data and puts it into a CSV file
for folder in os.listdir("dl_data/annotations"):
    dicom_path = f"dl_data/images/Lung_Dx-{folder}"
    annotation_path = f"dl_data/annotations/{folder}" 

    folder_list = ['A0001', 'A0002', 'A0003', 'A0004', 'A0005', 'B0004', 'B0005', 'E0001', 'E0002', 'G0004', 'G0005']
    
    if folder not in folder_list:
        continue
    else:
        print(folder)
    
    command = [
        "python", "preprocess.py", "--dicom-mode", "CT",
        "--dicom-path", dicom_path,
        "--annotation-path", annotation_path,
        "--classfile", "category.txt"
    ]
    
    # Execute the command
    subprocess.run(command)

E0001
Folder/File Found
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possible key error
Possibl

In [23]:
data = pd.read_csv('output.csv')
model, label_map, history = make_model(data['pixel_data'], data['cancer_type'])

Epoch 1/40
4/4 [==============================] - 2s 371ms/step - loss: 4.8095 - accuracy: 0.4519 - val_loss: 3004.6367 - val_accuracy: 0.3846
Epoch 2/40
4/4 [==============================] - 1s 333ms/step - loss: 0.7225 - accuracy: 0.8173 - val_loss: 2508.3896 - val_accuracy: 0.3846
Epoch 3/40
4/4 [==============================] - 1s 304ms/step - loss: 0.3920 - accuracy: 0.9423 - val_loss: 1242.2587 - val_accuracy: 0.3846
Epoch 4/40
4/4 [==============================] - 1s 305ms/step - loss: 0.0921 - accuracy: 0.9615 - val_loss: 804.7913 - val_accuracy: 0.5769
Epoch 5/40
4/4 [==============================] - 1s 302ms/step - loss: 0.2063 - accuracy: 0.9904 - val_loss: 611.8529 - val_accuracy: 0.3846
Epoch 6/40
4/4 [==============================] - 1s 307ms/step - loss: 1.9147e-04 - accuracy: 1.0000 - val_loss: 676.8092 - val_accuracy: 0.3846
Epoch 7/40
4/4 [==============================] - 1s 309ms/step - loss: 3.3831e-05 - accuracy: 1.0000 - val_loss: 653.1523 - val_accuracy: 0.